In [1]:
from openai import OpenAI
import os
from dotenv import load_dotenv
load_dotenv()

client = OpenAI( api_key=os.environ.get("OPENAI_API_KEY"))

In [2]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults(max_results=2,api_key=os.environ.get("TAVILY_API_KEY"),include_domains = ['www.healthyschoolrecipes.com'])
search_results = search.invoke("what are the ingredients recipe for Stromboli")

In [3]:
def make_system_prompt() -> str:
    return (
        "You are a helpful AI assistant, collaborating with other assistants."
        " Use the provided tools to progress towards answering the question."
        " Do NOT answer based on your own knowledge or assumptions."
        " ONLY respond if the tool provides a valid and relevant result."
        " If the tool fails or returns nothing useful, say you cannot find the answer."
        " If you or any of the other assistants have the final answer or deliverable,"
        " prefix your response with FINAL ANSWER so the team knows to stop."
    )

In [4]:
from langgraph.prebuilt import create_react_agent
from langchain.chat_models import init_chat_model

model = init_chat_model("gpt-4o", model_provider="openai",api_key=os.environ.get("OPENAI_API_KEY"))
search = TavilySearchResults(max_results=2,include_domains = ['www.healthyschoolrecipes.com'])
tools = [search]

agent_executor = create_react_agent(model, tools,
                                    prompt=make_system_prompt)

In [23]:
import os
from langchain.chat_models import ChatOpenAI
from langchain.tools.tavily_search import TavilySearchResults
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import HumanMessage

# 1. Setup Tavily tool
search = TavilySearchResults(
    max_results=2,
    include_domains=["www.healthyschoolrecipes.com"]  # Optional
)

# 2. Define system prompt
def make_system_prompt():
    return (
        "You are a helpful AI assistant, collaborating with other assistants."
        " Use the provided tools to progress towards answering the question."
        " Do NOT answer based on your own knowledge or assumptions."
        " ONLY respond if the tool provides a valid and relevant result."
        " If the tool fails or returns nothing useful, say you cannot find the answer."
        " If you or any of the other assistants have the final answer or deliverable,"
        " prefix your response with FINAL ANSWER so the team knows to stop."
         )

# 3. Setup LLM and agent
# llm = ChatOpenAI(model="gpt-4o", temperature=0)
model = init_chat_model("gpt-4o", model_provider="openai",api_key=os.environ.get("OPENAI_API_KEY"))
# Create agent with tool
agent_executor = create_react_agent(
    model,
    tools=[search],
    prompt=make_system_prompt()
)

# 4. Run query
query = "Fetch all the detailed ingredients for any idly recipe?"
response = agent_executor.invoke(
    {"messages": [HumanMessage(content=query)]}
)

print(response["messages"][-1].content)

I couldn't find any specific information about the detailed ingredients for an idly recipe.


In [24]:
from pydantic import BaseModel, Field
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

from typing import Literal

class RouteQuery(BaseModel):
    """Binary score to assess answer addresses question."""

    binary_score: Literal["yes", "no"] = Field(
        ...,
        description="Given a user question choose to route it to yes or a no.",
    )

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
structured_llm_grader = llm.with_structured_output(RouteQuery)
system = """You are a grader assessing whether the answer addresses / resolves a question \n 
     Give a binary score 'yes' or 'no'. Yes' means that the answer resolves the question."""
answer_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "User question: \n\n {question} \n\n LLM generation: {generation}"),
    ]
)

answer_grader = answer_prompt | structured_llm_grader
answer_grader.invoke({"question": query, "generation": response["messages"][-1].content})

RouteQuery(binary_score='no')

In [ ]:
from typing import List
from typing_extensions import TypedDict
from langgraph.graph import END, StateGraph, START


class GraphState(TypedDict):
    """
    Represents the state of our graph.

    Attributes:
        question: question
        generation: LLM generation
        documents: list of documents
    """

    question: str
    generation: str
    documents: List[str]

workflow = StateGraph(GraphState)
